# Customer Churn Prediction with XGBoost
_**Using Gradient Boosted Trees to Predict Mobile Customer Departure**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Compile](#Compile)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Relative cost of errors](#Relative-cost-of-errors)
1. [Extensions](#Extensions)

---

## Background

_This notebook has been adapted from an [AWS blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/)_

Losing customers is costly for any business.  Identifying unhappy customers early on gives you a chance to offer them incentives to stay.  This notebook describes using machine learning (ML) for the automated identification of unhappy customers, also known as customer churn prediction. ML models rarely give perfect predictions though, so this notebook is also about how to incorporate the relative costs of prediction mistakes when determining the financial outcome of using ML.

We use an example of churn that is familiar to all of us–leaving a mobile phone operator.  Seems like I can always find fault with my provider du jour! And if my provider knows that I’m thinking of leaving, it can offer timely incentives–I can always use a phone upgrade or perhaps have a new feature activated–and I might just stick around. Incentives are often much more cost effective than losing and reacquiring a customer.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [2]:
bucket = 'demo-saeed-ohio'
prefix = 'sagemaker/DEMO-xgboost-churn'

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

Next, we'll import the Python libraries we'll need for the remainder of the exercise.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

---
## Data

Mobile operators have historical records on which customers ultimately ended up churning and which continued using the service. We can use this historical information to construct an ML model of one mobile operator’s churn using a process called training. After training the model, we can pass the profile information of an arbitrary customer (the same profile information that we used to train the model) to the model, and have the model predict whether this customer is going to churn. Of course, we expect the model to make mistakes–after all, predicting the future is tricky business! But I’ll also show how to deal with prediction errors.

The dataset we use is publicly available and was mentioned in the book [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) by Daniel T. Larose. It is attributed by the author to the University of California Irvine Repository of Machine Learning Datasets.  Let's download and read that dataset in now:

In [4]:
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip

--2020-02-13 19:30:59--  http://dataminingconsultant.com/DKD2e_data_sets.zip
Resolving dataminingconsultant.com (dataminingconsultant.com)... 160.153.91.162
Connecting to dataminingconsultant.com (dataminingconsultant.com)|160.153.91.162|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1003616 (980K) [application/zip]
Saving to: ‘DKD2e_data_sets.zip’

DKD2e_data_sets.zip 100%[===================>] 980.09K  2.62MB/s    in 0.4s    

2020-02-13 19:30:59 (2.62 MB/s) - ‘DKD2e_data_sets.zip’ saved [1003616/1003616]

Archive:  DKD2e_data_sets.zip
 extracting: Data sets/adult.zip     
  inflating: Data sets/cars.txt      
  inflating: Data sets/cars2.txt     
  inflating: Data sets/cereals.CSV   
  inflating: Data sets/churn.txt     
  inflating: Data sets/ClassifyRisk  
  inflating: Data sets/ClassifyRisk - Missing.txt  
 extracting: Data sets/DKD2e data sets.zip  
  inflating: Data sets/nn1.txt       


In [104]:
churn = pd.read_csv('./Data sets/churn.txt')
pd.set_option('display.max_columns', 500)
churn.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


By modern standards, it’s a relatively small dataset, with only 3,333 records, where each record uses 21 attributes to describe the profile of a customer of an unknown US mobile operator. The attributes are:

- `State`: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- `Account Length`: the number of days that this account has been active
- `Area Code`: the three-digit area code of the corresponding customer’s phone number
- `Phone`: the remaining seven-digit phone number
- `Int’l Plan`: whether the customer has an international calling plan: yes/no
- `VMail Plan`: whether the customer has a voice mail feature: yes/no
- `VMail Message`: presumably the average number of voice mail messages per month
- `Day Mins`: the total number of calling minutes used during the day
- `Day Calls`: the total number of calls placed during the day
- `Day Charge`: the billed cost of daytime calls
- `Eve Mins, Eve Calls, Eve Charge`: the billed cost for calls placed during the evening
- `Night Mins`, `Night Calls`, `Night Charge`: the billed cost for calls placed during nighttime
- `Intl Mins`, `Intl Calls`, `Intl Charge`: the billed cost for international calls
- `CustServ Calls`: the number of calls placed to Customer Service
- `Churn?`: whether the customer left the service: true/false

The last attribute, `Churn?`, is known as the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction, also known as binary classification.

Let's begin exploring the data:

In [179]:
#churn.to_csv('Data sets/train.csv',header=None, index=False)

In [180]:
!pwd

/home/ec2-user/SageMaker/xgboost_customer_churn_2020-02-13


And now let's split the data into training, validation, and test sets.  This will help prevent us from overfitting the model, and allow us to test the models accuracy on data it hasn't already seen.

In [296]:
train_data, validation_data, test_data = np.split(churn.sample(frac=1, random_state=1729), [int(0.7 * len(churn)), int(0.9 * len(churn))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)
test_data.drop('Churn?', axis=1).to_csv('test.csv', header=False, index=False)

Now we'll upload these files to S3.

In [297]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'rawtrain/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'rawvalidation/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'rawtest/test.csv')).upload_file('test.csv')


In [318]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/rawtrain/'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/rawvalidation/'.format(bucket, prefix), content_type='csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/rawtest/'.format(bucket, prefix), content_type='csv')

## Train the transformer

In [359]:
from sagemaker.sklearn.estimator import SKLearn
sagemaker_session = sagemaker.Session()
script_path = 'sk.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    train_instance_type="local")
sklearn_preprocessor.fit({'train': s3_input_train})

Creating tmp6le8q1xn_algo-1-nezws_1 ... 
Attaching to tmp6le8q1xn_algo-1-nezws_12mdone
algo-1-nezws_1  | 2020-02-16 05:45:30,143 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-nezws_1  | 2020-02-16 05:45:30,147 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-nezws_1  | 2020-02-16 05:45:30,159 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-nezws_1  | 2020-02-16 05:45:30,290 sagemaker-containers INFO     Module sk does not provide a setup.py. 
algo-1-nezws_1  | Generating setup.py
algo-1-nezws_1  | 2020-02-16 05:45:30,290 sagemaker-containers INFO     Generating setup.cfg
algo-1-nezws_1  | 2020-02-16 05:45:30,291 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-nezws_1  | 2020-02-16 05:45:30,291 sagemaker-containers INFO     Installing module with the following command:
algo-1-nezws_1  | /miniconda3/bin/python -m pip install . 
algo-1-nezws_1  | Processing /op

## Transform

In [360]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transform_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-train-output')
transformer_train = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='local',
    assemble_with = 'Line',
    output_path = transform_output_path,
    accept = 'text/csv')
# Preprocess training input
transformer_train.transform('s3://demo-saeed-ohio/sagemaker/DEMO-xgboost-churn/rawtrain/train.csv', content_type='text/csv')
print('Waiting for transform job: ' + transformer_train.latest_transform_job.job_name)
transformer_train.wait()
preprocessed_train_path = transformer_train.output_path+transformer_train.latest_transform_job.job_name
print(preprocessed_train_path)


Attaching to tmp85l9u8jb_algo-1-yxnyv_1
algo-1-yxnyv_1  | Processing /opt/ml/code
algo-1-yxnyv_1  | Building wheels for collected packages: sk
algo-1-yxnyv_1  |   Building wheel for sk (setup.py) ... done
algo-1-yxnyv_1  |   Created wheel for sk: filename=sk-1.0.0-py2.py3-none-any.whl size=9655 sha256=3b1c1fa14f70e9f545eed3a76583ea4f29f1100db1981ef7fc4e79969c3fa4d8
algo-1-yxnyv_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-nosv0qsn/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-yxnyv_1  | Successfully built sk
algo-1-yxnyv_1  | Installing collected packages: sk
algo-1-yxnyv_1  | Successfully installed sk-1.0.0
algo-1-yxnyv_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-yxnyv_1  |   import imp
algo-1-yxnyv_1  | [2020-02-16 05:45:47 +0000] [30] [INFO] Starting

In [361]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transform_validation_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-validation-output')
transformer_validation = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='local',
    assemble_with = 'Line',
    output_path = transform_validation_output_path,
    accept = 'text/csv')
# Preprocess training input
transformer_validation.transform('s3://demo-saeed-ohio/sagemaker/DEMO-xgboost-churn/rawvalidation/validation.csv', content_type='text/csv')
print('Waiting for transform job: ' + transformer_validation.latest_transform_job.job_name)
transformer_validation.wait()
preprocessed_validation_path = transformer_validation.output_path+transformer_validation.latest_transform_job.job_name
print(preprocessed_validation_path)


Attaching to tmpuy3_8fcc_algo-1-il81n_1
algo-1-il81n_1  | Processing /opt/ml/code
algo-1-il81n_1  | Building wheels for collected packages: sk
algo-1-il81n_1  |   Building wheel for sk (setup.py) ... done
algo-1-il81n_1  |   Created wheel for sk: filename=sk-1.0.0-py2.py3-none-any.whl size=9654 sha256=164879adc3d5fa00617de6223b9bfb8a84fcb7315aef8ef26857adc83df64ed5
algo-1-il81n_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-wwglbasg/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-il81n_1  | Successfully built sk
algo-1-il81n_1  | Installing collected packages: sk
algo-1-il81n_1  | Successfully installed sk-1.0.0
algo-1-il81n_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-il81n_1  |   import imp
algo-1-il81n_1  | [2020-02-16 05:46:06 +0000] [30] [INFO] Starting

In [362]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transform_test_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-test-output')
transformer_test = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='local',
    assemble_with = 'Line',
    output_path = transform_test_output_path,
    accept = 'text/csv')
# Preprocess training input
transformer_test.transform('s3://demo-saeed-ohio/sagemaker/DEMO-xgboost-churn/rawtest/test.csv', content_type='text/csv')
print('Waiting for transform job: ' + transformer_test.latest_transform_job.job_name)
transformer_test.wait()
preprocessed_test_path = transformer_test.output_path+transformer_test.latest_transform_job.job_name
print(preprocessed_test_path)


Attaching to tmpuhx93sur_algo-1-3gncp_1
algo-1-3gncp_1  | Processing /opt/ml/code
algo-1-3gncp_1  | Building wheels for collected packages: sk
algo-1-3gncp_1  |   Building wheel for sk (setup.py) ... done
algo-1-3gncp_1  |   Created wheel for sk: filename=sk-1.0.0-py2.py3-none-any.whl size=9653 sha256=a9ad067365af0c6f29b7a4227a5ca7851feca01874ac6798f53be2e4f47cbb23
algo-1-3gncp_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-9n2usl8_/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-3gncp_1  | Successfully built sk
algo-1-3gncp_1  | Installing collected packages: sk
algo-1-3gncp_1  | Successfully installed sk-1.0.0
algo-1-3gncp_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-3gncp_1  |   import imp
algo-1-3gncp_1  | [2020-02-16 05:46:13 +0000] [30] [INFO] Starting

---
## Train

Moving onto training, first we'll need to specify the locations of the XGBoost algorithm containers.

In [311]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')

Then, because we're training with the CSV file format, we'll create `s3_input`s that our training function can use as a pointer to the files in S3.

In [347]:
s3_input_train_processed = sagemaker.session.s3_input(
    preprocessed_train_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print(s3_input_train_processed.config)
s3_input_validation_processed = sagemaker.session.s3_input(
    preprocessed_validation_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print(s3_input_validation_processed.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://demo-saeed-ohio/sagemaker/DEMO-xgboost-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-02-16-03-56-35-170', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://demo-saeed-ohio/sagemaker/DEMO-xgboost-churn/transformtrain-validation-output/sagemaker-scikit-learn-2020-02-16-03-56-50-532', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}


Now, we can specify a few parameters like what type of training instances we'd like to use and how many, as well as our XGBoost hyperparameters.  A few key hyperparameters are:
- `max_depth` controls how deep each tree within the algorithm can be built.  Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting.  There is typically some trade-off in model performance that needs to be explored between a large number of shallow trees and a smaller number of deeper trees.
- `subsample` controls sampling of the training data.  This technique can help reduce overfitting, but setting it too low can also starve the model of data.
- `num_round` controls the number of boosting rounds.  This is essentially the subsequent models that are trained using the residuals of previous iterations.  Again, more rounds should produce a better fit on the training data, but can be computationally expensive or lead to overfitting.
- `eta` controls how aggressive each round of boosting is.  Larger values lead to more conservative boosting.
- `gamma` controls how aggressively trees are grown.  Larger values lead to more conservative models.

More detail on XGBoost's hyperparmeters can be found on their GitHub [page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md).

In [348]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train_processed, 'validation': s3_input_validation_processed}) 

2020-02-16 03:58:09 Starting - Starting the training job...
2020-02-16 03:58:10 Starting - Launching requested ML instances...
2020-02-16 03:59:08 Starting - Preparing the instances for training.........
2020-02-16 04:00:29 Downloading - Downloading input data
2020-02-16 04:00:29 Training - Downloading the training image...
2020-02-16 04:01:05 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[04:01:01] 2333x69 matrix with 160977 entries loaded from /opt/ml/input/data/train?forma

In [349]:
transform_xgb_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-xgb-output')

# Define a SKLearn Transformer from the trained SKLearn Estimator
xgb_transformer = xgb.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    output_path = transform_xgb_output_path,
    assemble_with = 'Line',
    accept = 'text/csv')

In [350]:
s3_input_test_processed = sagemaker.session.s3_input(
    preprocessed_test_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print(s3_input_test_processed.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://demo-saeed-ohio/sagemaker/DEMO-xgboost-churn/transformtrain-test-output/sagemaker-scikit-learn-2020-02-16-03-57-08-395', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}


In [352]:
# Preprocess training input
xgb_transformer.transform(preprocessed_test_path, content_type='text/csv')
print('Waiting for transform job: ' + xgb_transformer.latest_transform_job.job_name)
xgb_transformer.wait()
xgb_transformer.output_path

Waiting for transform job: sagemaker-xgboost-2020-02-16-04-08-07-064
.......................[2020-02-16 04:11:40 +0000] [15] [INFO] Starting gunicorn 19.10.0
[2020-02-16 04:11:40 +0000] [15] [INFO] Listening at: unix:/tmp/gunicorn.sock (15)
[2020-02-16 04:11:40 +0000] [15] [INFO] Starting gunicorn 19.10.0
[2020-02-16 04:11:40 +0000] [15] [INFO] Listening at: unix:/tmp/gunicorn.sock (15)
[2020-02-16 04:11:40 +0000] [15] [INFO] Using worker: gevent
[2020-02-16 04:11:40 +0000] [22] [INFO] Booting worker with pid: 22
[2020-02-16 04:11:40 +0000] [23] [INFO] Booting worker with pid: 23
[2020-02-16 04:11:41 +0000] [27] [INFO] Booting worker with pid: 27
[2020-02-16 04:11:41 +0000] [28] [INFO] Booting worker with pid: 28
[2020-02-16 04:11:40 +0000] [15] [INFO] Using worker: gevent
[2020-02-16 04:11:40 +0000] [22] [INFO] Booting worker with pid: 22
[2020-02-16 04:11:40 +0000] [23] [INFO] Booting worker with pid: 23
[2020-02-16 04:11:41 +0000] [27] [INFO] Booting worker with pid: 27
[2020-02-16 

's3://demo-saeed-ohio/sagemaker/DEMO-xgboost-churn/transformtrain-xgb-output/'

## Pipeline

In [363]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
xgboost_model = xgb.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
#endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, 
    role=role, 
    models=[
        scikit_learn_inferencee_model, 
        xgboost_model])

#sm_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

In [364]:
sm_model

In [365]:
model_name

'inference-pipeline-2020-02-16-05-46-27'

In [367]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transform_pipeline_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-pipeline-output')
pipeline_transformer = sm_model.transformer(
    instance_count=1, 
    output_path = transform_pipeline_output_path,
    instance_type='ml.m4.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv') 

In [368]:
# Preprocess training input
pipeline_transformer.transform('s3://demo-saeed-ohio/sagemaker/DEMO-xgboost-churn/rawtest/test.csv', content_type='text/csv')
print('Waiting for transform job: ' + pipeline_transformer.latest_transform_job.job_name)
transformer.wait()
pipeline_transformer.output_path

Waiting for transform job: inference-pipeline-2020-02-16-05-46-27-2020-02-16-05-47-17-936
.

's3://demo-saeed-ohio/sagemaker/DEMO-xgboost-churn/transformtrain-pipeline-output/'

---
## Compile
[Amazon SageMaker Neo](https://aws.amazon.com/sagemaker/neo/) optimizes models to run up to twice as fast, with no loss in accuracy. When calling `compile_model()` function, we specify the target instance family (m4) as well as the S3 bucket to which the compiled model would be stored.

In [ ]:
compiled_model = xgb
try:
    xgb.create_model()._neo_image_account(boto3.Session().region_name)
except:
    print('Neo is not currently supported in', boto3.Session().region_name)
else:
    output_path = '/'.join(xgb.output_path.split('/')[:-1])
    compiled_model = xgb.compile_model(target_instance_family='ml_m4', 
                                   input_shape={'data':[1, 69]},
                                   role=role,
                                   framework='xgboost',
                                   framework_version='0.7',
                                   output_path=output_path)
    compiled_model.name = 'deployed-xgboost-customer-churn'
    compiled_model.image = get_image_uri(sess.boto_region_name, 'xgboost-neo', repo_version='latest')

---
## Host

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

In [ ]:
xgb_predictor = compiled_model.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

### Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request.  But first, we'll need to setup serializers and deserializers for passing our `test_data` NumPy arrays to the model behind the endpoint.

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batchs to CSV string payloads
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.as_matrix()[:, 1:])

There are many ways to compare the performance of a machine learning model, but let's start by simply by comparing actual to predicted values.  In this case, we're simply predicting whether the customer churned (`1`) or not (`0`), which produces a simple confusion matrix.

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])

_Note, due to randomized elements of the algorithm, you results may differ slightly._

Of the 48 churners, we've correctly predicted 39 of them (true positives). And, we incorrectly predicted 4 customers would churn who then ended up not doing so (false positives).  There are also 9 customers who ended up churning, that we predicted would not (false negatives).

An important point here is that because of the `np.round()` function above we are using a simple threshold (or cutoff) of 0.5.  Our predictions from `xgboost` come out as continuous values between 0 and 1 and we force them into the binary classes that we began with.  However, because a customer that churns is expected to cost the company more than proactively trying to retain a customer who we think might churn, we should consider adjusting this cutoff.  That will almost certainly increase the number of false positives, but it can also be expected to increase the number of true positives and reduce the number of false negatives.

To get a rough intuition here, let's look at the continuous values of our predictions.

In [ ]:
plt.hist(predictions)
plt.show()

The continuous valued predictions coming from our model tend to skew toward 0 or 1, but there is sufficient mass between 0.1 and 0.9 that adjusting the cutoff should indeed shift a number of customers' predictions.  For example...

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.where(predictions > 0.3, 1, 0))

We can see that changing the cutoff from 0.5 to 0.3 results in 1 more true positives, 3 more false positives, and 1 fewer false negatives.  The numbers are small overall here, but that's 6-10% of customers overall that are shifting because of a change to the cutoff.  Was this the right decision?  We may end up retaining 3 extra customers, but we also unnecessarily incentivized 5 more customers who would have stayed.  Determining optimal cutoffs is a key step in properly applying machine learning in a real-world setting.  Let's discuss this more broadly and then apply a specific, hypothetical solution for our current problem.

### Relative cost of errors

Any practical binary classification problem is likely to produce a similarly sensitive cutoff. That by itself isn’t a problem. After all, if the scores for two classes are really easy to separate, the problem probably isn’t very hard to begin with and might even be solvable with simple rules instead of ML.

More important, if I put an ML model into production, there are costs associated with the model erroneously assigning false positives and false negatives. I also need to look at similar costs associated with correct predictions of true positives and true negatives.  Because the choice of the cutoff affects all four of these statistics, I need to consider the relative costs to the business for each of these four outcomes for each prediction.

#### Assigning costs

What are the costs for our problem of mobile operator churn? The costs, of course, depend on the specific actions that the business takes. Let's make some assumptions here.

First, assign the true negatives the cost of \$0. Our model essentially correctly identified a happy customer in this case, and we don’t need to do anything.

False negatives are the most problematic, because they incorrectly predict that a churning customer will stay. We lose the customer and will have to pay all the costs of acquiring a replacement customer, including foregone revenue, advertising costs, administrative costs, point of sale costs, and likely a phone hardware subsidy. A quick search on the Internet reveals that such costs typically run in the hundreds of dollars so, for the purposes of this example, let's assume \$500. This is the cost of false negatives.

Finally, for customers that our model identifies as churning, let's assume a retention incentive in the amount of \$100. If my provider offered me such a concession, I’d certainly think twice before leaving. This is the cost of both true positive and false positive outcomes. In the case of false positives (the customer is happy, but the model mistakenly predicted churn), we will “waste” the \$100 concession. We probably could have spent that \$100 more effectively, but it's possible we increased the loyalty of an already loyal customer, so that’s not so bad.

#### Finding the optimal cutoff

It’s clear that false negatives are substantially more costly than false positives. Instead of optimizing for error based on the number of customers, we should be minimizing a cost function that looks like this:

```txt
$500 * FN(C) + $0 * TN(C) + $100 * FP(C) + $100 * TP(C)
```

FN(C) means that the false negative percentage is a function of the cutoff, C, and similar for TN, FP, and TP.  We need to find the cutoff, C, where the result of the expression is smallest.

A straightforward way to do this, is to simply run a simulation over a large number of possible cutoffs.  We test 100 possible values in the for loop below.

In [ ]:
cutoffs = np.arange(0.01, 1, 0.01)
costs = []
for c in cutoffs:
    costs.append(np.sum(np.sum(np.array([[0, 100], [500, 100]]) * 
                               pd.crosstab(index=test_data.iloc[:, 0], 
                                           columns=np.where(predictions > c, 1, 0)))))

costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.show()
print('Cost is minimized near a cutoff of:', cutoffs[np.argmin(costs)], 'for a cost of:', np.min(costs))

The above chart shows how picking a threshold too low results in costs skyrocketing as all customers are given a retention incentive.  Meanwhile, setting the threshold too high results in too many lost customers, which ultimately grows to be nearly as costly.  The overall cost can be minimized at \$8400 by setting the cutoff to 0.46, which is substantially better than the \$20k+ I would expect to lose by not taking any action.

---
## Extensions

This notebook showcased how to build a model that predicts whether a customer is likely to churn, and then how to optimally set a threshold that accounts for the cost of true positives, false positives, and false negatives.  There are several means of extending it including:
- Some customers who receive retention incentives will still churn.  Including a probability of churning despite receiving an incentive in our cost function would provide a better ROI on our retention programs.
- Customers who switch to a lower-priced plan or who deactivate a paid feature represent different kinds of churn that could be modeled separately.
- Modeling the evolution of customer behavior. If usage is dropping and the number of calls placed to Customer Service is increasing, you are more likely to experience churn then if the trend is the opposite. A customer profile should incorporate behavior trends.
- Actual training data and monetary cost assignments could be more complex.
- Multiple models for each type of churn could be needed.

Regardless of additional complexity, similar principles described in this notebook are likely apply.

### (Optional) Clean-up

If you're ready to be done with this notebook, please run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)